In [3]:
import pandas as pd
import numpy as np
from scipy import stats

In [17]:
data_mutation = pd.read_csv('csv_data/mutation_latents.csv', index_col='filename')
data_no_mutation = pd.read_csv('csv_data/no_mutation_latents.csv', index_col='filename')

# Используются:

# Shapiro-Wilk тест для нормальности
# Levene тест для равных дисперсий
# Kolmogorov-Smirnov для сравнения распределений
# Student-t тест для сравнения средних распределений
# Mann-Whitney тест для сравнения "медиан" распределений

In [18]:
data_mutation['label'] = 1
data_no_mutation['label'] = 0
data = pd.concat((data_mutation, data_no_mutation))
data.reset_index(inplace=True)

In [19]:
numerical = data_mutation.columns

In [20]:
global_tests_p_value = 0.1

In [21]:
def check_normality(data):
    _, p_value = stats.shapiro(data)
    return p_value > 0.05

In [22]:
def check_variance(data_1, data_2):
    _, p_value = stats.levene(data_1, data_2)
    return p_value > 0.05

In [23]:
def pearson_coef(data_1, data_2, cat):
    data = pd.concat((data_1, data_2))
    results = stats.s

In [24]:
def ks_2sample_test(data_1, data_2, cat):
    results = stats.ks_2samp(data_1, data_2, method='exact')
    return {
                'Факторы': cat,
                'P_ks_test': f'{results.pvalue:.4f}',
                'Difference_ks': f'{True if results.pvalue < global_tests_p_value else False}'
            }

In [25]:
def mann_whitney_test(data_1, data_2, cat):
    if (not check_normality(data_1)) or (not check_normality(data_2)):
        results = stats.mannwhitneyu(data_1, data_2, method='exact')
        return {
                    'Факторы': cat,
                    'Мутация_медиана': f'{data_1.median():.3f}',
                    'Нет мутации_медиана': f'{data_2.median():.3f}',
                    'P_mw_test': f'{results.pvalue:.4f}',
                    'Difference_mw': f'{True if results.pvalue < global_tests_p_value else False}'
                }
    else:
        return {
                    'Факторы': cat,
                    'Мутация_медиана': f'{data_1.median():.3f}',
                    'Нет мутации_медиана': f'{data_2.median():.3f}',
                    'P_mw_test': '',
                    'Difference_mw': ''
                }

In [26]:
def t_test(data_1, data_2, cat):
    if check_variance(data_1, data_2):
        if check_normality(data_1) and check_normality(data_2):
            results = stats.ttest_ind(data_1, data_2, alternative='two-sided')
            return {
                'Факторы': cat,
                'Мутация_среднее': f'{data_1.mean():.3f}',
                'Нет мутации_среднее': f'{data_2.mean():.3f}',
                'P_student_test': f'{results.pvalue:.4f}',
                'Difference_student': f'{True if results.pvalue < global_tests_p_value else False}'
            }
        else:
            return {
                'Факторы': cat,
                'Мутация_среднее': f'{data_1.mean():.3f}',
                'Нет мутации_среднее': f'{data_2.mean():.3f}',
                'P_student_test': '',
                'Difference_student': ''
            }
    else:
        if check_normality(data_1) and check_normality(data_2):
            results = stats.ttest_ind(data_1, data_2, equal_var=False, alternative='two-sided')
            return {
                'Факторы': cat,
                'Мутация_среднее': f'{data_1.mean():.3f}',
                'Нет мутации_среднее': f'{data_2.mean():.3f}',
                'P_student_test': f'{results.pvalue:.4f}',
                'Difference_student': f'{True if results.pvalue < global_tests_p_value else False}'
            }
        else:
            return {
                'Факторы': cat,
                'Мутация_среднее': f'{data_1.mean():.3f}',
                'Нет мутации_среднее': f'{data_2.mean():.3f}',
                'P_student_test': '',
                'Difference_student': ''
            }

In [27]:
results_data = []

for cat in numerical:
    results_t = t_test(data_mutation[cat], data_no_mutation[cat], cat)
    results_ks = ks_2sample_test(data_mutation[cat], data_no_mutation[cat], cat)
    results_mw = mann_whitney_test(data_mutation[cat], data_no_mutation[cat], cat)
    
    results_t.update(results_ks)
    results_t.update(results_mw)
    
    results_data.append(results_t)
results_df = pd.DataFrame(results_data)


c:\Users\kungu\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:3345: RuntimeWarning: invalid value encountered in scalar divide
  W = numer / denom
c:\Users\kungu\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\kungu\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:3345: RuntimeWarning: invalid value encountered in scalar divide
  W = numer / denom
c:\Users\kungu\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\kungu\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:3345: RuntimeWarning: invalid value enc

In [29]:
excel_file = 'features_difference_latents.xlsx'
with pd.ExcelWriter(excel_file) as writer:
    results_df.to_excel(writer, sheet_name='Сравнение групп', index=False)